# Practical 7 - Part 2A
This second half of the lab explores the geometry of a single camera. In 2B the goal is to use a set of correspondance points to estimate a transformation matrix from a plane's 3D space to camera space and use that matrix to project some other points into camera space.

In this section, we'll work on building two components that we need for 2B, a method to estimate that transformation and a method that can project points into camera image space.

First we'll tackle the projection method, `projectiveCamera`. We want to find the image space coordinates, `XImCart`, of a set of 3D world coordinates, `XCart`, given a camera intrinsics matrix `K` and an extrinsics matrix `T`.

The second component is a method to estimate a Eucledian transformation, `TEst`, that takes us from a plane's 3D coordinate space to 3D camera space by utilizing a given set of points in camera image space, `XImCart`, and a set of corresponding points in world space, `XCart`. Essentially we want to compute the extrinsics matrix we can use in `projectiveCamera`.

Estimating the camera pose will involve calculating a homography, so you'll need to copy over your functions from part 1A/1B in the space provided.

## Import libraries 

In [1]:
%matplotlib inline
import os 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

In [13]:
def projectiveCamera(K, T, XCart):
    ##TODO
    # The goal of this function is to project points in XCart through projective camera
    # defined by intrinsic matrix K and extrinsic matrix T. In essence, this function takes a set of points 
    # in 3D world space, XCart, and projects them into camera image space by applying the extrinsic matrix T 
    # and then applying the intrinsic matrix K.
    # 
    # There are three steps.
    # 1) Move from world space to camera space. 
    #            camera space points = extrinsics T * world space points 
    # 2) Applying the intrinsics matrix to the camera space points after normalizing
    #           homogeneous image space points = K * normalized camera space points
    # 3) Move to image space cartesian points from image space homogeneous points, involves a 
    # normalization using the third row.
    
    
    # TO DO: Replace this
    #XImCart = []

    # TO DO: Convert Cartesian 3d points XCart to homogeneous coordinates XHom
    XHom = np.concatenate((XCart, np.ones((1, XCart.shape[1]))), axis=0)
    
    # TO DO: Apply extrinsic matrix to XHom, to move to frame of reference of camera
    XHom = T @ XHom
    
    # TO DO: Project points into normalized camera coordinates xCamHom (remove 4th row)
    xCamHom = XHom[:3]
    
    # TO DO: Move points to image coordinates xImHom by applying intrinsic matrix
    xImHom = K @ xCamHom
    
    # TO DO: Convert points back to Cartesian coordinates xImCart
    XImCart = xImHom[:2] / xImHom[2]
    
    return XImCart


## Camera Projection

First we'll write up the function that can take us from 3D world space, `XCart`, to camera image space, `XImCart`, using an extrinsics matrix `T` and an intrinsics matrix `K` that are provided. The previous block houses this function.

The result here is the cartesian image space point coordinates, `XImCart`, of the 3D points `XCart`. If `XCart` represents a box in the world then we now know where the box's vertices would land in image space.

To verify that your solution is correct please compare your image space points to those in the comment.

Once they match, move on to the next bit - estimating a transformation! 

In [14]:
# We assume that the camera intrinsics matrix K is known and has values:
K = np.array([[640, 0, 320],
             [0, 640, 240],
             [0, 0, 1]])

# We will assume an object co-ordinate system with the Z-axis pointing upwards and the origin
# in the centre of the plane. There are five known points on the plane with coordinates (mm):
XCart = np.array([[-100, -100,  100,  100, 0],
                  [-100,  100,  100, -100, 0],
                  [   0,    0,    0,    0, 0]])

# We assume the correct transformation from the plane co-ordinate system to the
# camera co-ordinate system (extrinsic matrix) is:
T = np.array([[0.9851,  -0.0492,  0.1619,  46.00],
             [-0.1623,  -0.5520,  0.8181,  70.00],
             [0.0490,  -0.8324, -0.5518,  500.89],
             [0,        0,       0,       1]])
# T houses a rotation matrix and a translation matrix. The last row is for homogeneous point calculation.


# TO DO: Use the general pin-hole projective camera model discussed in the lectures to estimate 
# where the four points on the plane will appear in the image.  Fill in the
# details of the function "projectiveCamera"
XImCart = projectiveCamera(K, T, XCart)

print(XImCart)
# Should be around:
# [267.4170882  230.95045427 531.42492013 482.36049098 378.77537982]
# [396.26814909 288.11435494 237.83410247 358.39940241 329.44079538]

[[267.4170882  230.95045427 531.42492013 482.36049098 378.77537982]
 [396.26814909 288.11435494 237.83410247 358.39940241 329.44079538]]


### You've implemented both of these functions in 1A and 1B already, so feel free to copy them in here. You'll need them for this next part.

In [20]:
def solveAXEqualsZero(A):
    U, L, V = np.linalg.svd(A)
    h = V[-1]
  
    return h

In [19]:
def calcBestHomography(pts1Cart, pts2Cart):
    
    # This function should apply the direct linear transform (DLT) algorithm to calculate the best 
    # homography that maps the cartesian points in pts1Cart to their corresonding matching cartesian poitns 
    # in pts2Cart.
    
    # This function calls solveAXEqualsZero. Make sure you are wary of how to reshape h into a 3 by 3 matrix. 

    n_points = pts1Cart.shape[1]
    
    # TO DO: replace this:
    H = np.eye(3)

    # TO DO: 
    # First convert points into homogeneous representation
    # Hint: we've done this before  in the skeleton code we provide.
    pts1Hom = np.concatenate((pts1Cart, np.ones((1,pts1Cart.shape[1]))), axis=0)

    pts2Hom = np.concatenate((pts2Cart, np.ones((1,pts2Cart.shape[1]))), axis=0)

    
    A = np.zeros((n_points * 2, 9))
    j=0
    for i in range(0, n_points*2, 2):
        u = pts1Hom[0, j]
        v = pts1Hom[1, j]
        x = pts2Hom[0, j]
        y = pts2Hom[1, j]
        j = j+1
        A[i,:] = np.concatenate((np.array([0,0,0]), np.array([-u,-v,-1]), np.array([y*u,y*v,y])), axis=0)
        A[i+1,:] = np.concatenate((np.array([u,v,1]), np.array([0,0,0]), np.array([-x*u,-x*v,-x])), axis=0)
        
    # Solve Ah = 0 using solveAXEqualsZero and get h.
    H = solveAXEqualsZero(A).reshape(3,3)
  
    return H/H[2,2]

### Intrinsic matrix K
Intrinsic matrix K
$$
K = \begin{bmatrix}
f_x & s & c_x \\
0 & f_y & c_y \\
0 & 0 & 1
\end{bmatrix}
$$

To convert image coordinates to normalized camera coordinates, need to apply the inverse of the camera's intrinsic matrix `K` to the homogeneous image coordinates. 

Where `f_x` and `f_y` are the focal lengths in pixels, `c_x` and `c_y` are the coordinates of the principal point (usually the image center), and `s` is the skew coefficient.
Focal length, in the context of cameras and imaging, is the distance between the camera's lens and the image sensor when the subject of the photo is in focus. It is usually measured in millimeters (mm). 

Focal length determines the field of view of the camera (how much of the scene it can capture) and affects the perspective of the image. 

The normalized camera coordinates are the coordinates in the camera's image plane, but measured in a unit where the focal length is 1.

### Rotation matrix
To estimate the first two columns of the rotation matrix, we compute this singular value decomposition
$$
\begin{bmatrix}
\phi'_{11} & \phi'_{12} \\
\phi'_{21} & \phi'_{22} \\
\phi'_{31} & \phi'_{32}
\end{bmatrix}
= U \Sigma V^T
$$

Then we set
$$
\begin{bmatrix}
\omega_{11} & \omega_{12} \\
\omega_{21} & \omega_{22} \\
\omega_{31} & \omega_{32}
\end{bmatrix}
=
U
\begin{bmatrix}
1 & 0 \\
0 & 1 \\
0 & 0
\end{bmatrix}
V^T
$$

In [107]:
# Read the next cell first for context!

def estimatePlanePose(XImCart,XCart,K):
    # The goal of this function is to estimate the pose of a plane relative to camera (extrinsic matrix)
    # given points in image space xImCart, points in 3D world space XCart, and an intrinsics matrix K.
    
    # TO DO: replace this

    # TO DO: Convert Cartesian image points XImCart to homogeneous representation XImHom
    XImHom = np.concatenate((XImCart, np.ones((1, XImCart.shape[1]))), axis=0)
    
    
    # TO DO: Convert image co-ordinates XImHom to normalized camera coordinates XCamHom    
    XCamHom = np.linalg.inv(K) @ XImHom
    
    # TO DO: Estimate homography H mapping homogeneous (x,y) coordinates of positions
    # in real world to XCamHom (convert XCamHom to Cartesian, calculate the homography) -
    # use the routine you wrote for Practical 1B
    XCamCart = XCart[:2,:]
    XCamHom = XCamHom[:2,:]
    HEst = calcBestHomography(XCamCart, XCart)
    H = calcBestHomography(XCamCart, XCamHom)
          
    # TO DO: Estimate first two columns of rotation matrix R from the first two
    # columns of H using the SVD. NOTE: You do not need to transpose v from linalg.svd
    R = np.eye(3)
    U, L, V = np.linalg.svd(H[:,:2])
    R[:,:2] = U @ R[:,:2]@ V.T


    # TO DO: Estimate the third column of the rotation matrix by taking the cross
    # product of the first two columns
    R[:,2] = np.cross(R[:,0], R[:,1])
   
        
    # TO DO: Check that the determinant of the rotation matrix is positive - if
    # not then multiply last column by -1.
    assert np.linalg.det(R) > 0 
    if np.linalg.det(R) <= 0:
        R[:,2] = -R[:,2]

    
    # TO DO: Estimate the translation t by finding the appropriate scaling factor k
    # and applying it to the third colulmn of H
    # TODO: why is the sum over the first two columns of H and R
    k = np.sum(H[:,:2]/R[:,:2])/6
    t = H[:,2] / k    
    
    # TO DO: Check whether t_z is negative - if it is then multiply t by -1 and
    # the first two columns of R by -1.
    if t[-1] < 0:
        t = -t
        R[:,:2] = -R[:,:2]
            
    # TO DO: Assemble transformation into matrix form
    T = np.concatenate((R, t.reshape(3,1)), axis=1)
    T = np.concatenate((T, np.array([[0,0,0,1]])), axis=0)
    
    return T 

## Now the juicy bit: Estimating an Extrinsics Matrix, T

The problem: We are given an instrinsics matrix `K`, a set of 3D world points `XCart`, and a set of corresponding image space coordinates in `XImCart`. `K` and `XCart` have already been defined a few cells back and you've calculated `XImCart` by virtue of the exercise you've completed with camera projection. What we don't have is an extrinsics matrix, `T`. We need to estimate this and you'll need to fill in `estimatePlanePose` and return `TEst`.

Again you can start by negating the noise we add to XImCart to make sure you're on the right track.

In [109]:
# TO DO: Add noise (standard deviation of one pixel in each direction) to the pixel positions
# to simulate having to find these points in a noisy image. Store the results back in xImCart
XImCartNoisy = XImCart #add noise here

# TO DO: Now we will take the image points and the known positions on the card and estimate  
# the extrinsic matrix using the algorithm discussed in the lecture.  Fill in the details of 
# the function estimatePlanePose
TEst = estimatePlanePose(XImCartNoisy,XCart,K)

# If you have got this correct, TEst should closely resemble the groundtruth, T.
np.set_printoptions(precision=3)
print(TEst)
print("\n")
print(T)


[[ 1.967e-03 -9.823e-05  9.184e-02]
 [-3.240e-04 -1.102e-03  1.398e-01]
 [ 9.783e-05 -1.662e-03  1.000e+00]]
[[ 0.986 -0.049  0.162]
 [-0.162 -0.552  0.818]
 [ 0.049 -0.832 -0.552]]
0.0019941789475365814
[ 46.052  70.08  501.46 ]
[[ 9.855e-01 -4.936e-02  1.622e-01  4.605e+01]
 [-1.623e-01 -5.520e-01  8.179e-01  7.008e+01]
 [ 4.916e-02 -8.324e-01 -5.520e-01  5.015e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]


[[ 9.851e-01 -4.920e-02  1.619e-01  4.600e+01]
 [-1.623e-01 -5.520e-01  8.181e-01  7.000e+01]
 [ 4.900e-02 -8.324e-01 -5.518e-01  5.009e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]
